# Amazon Sagemakerを利用したきのこの山検出AIの実装研修

## 研修のゴール共有
たけのこの里ときのこの山の何れかがベルトコンベアーで流れて来たときに、物体を認識させきのこの山を検出する。

## 目次

1. モデル学習用の画像を撮影する
    * S3バケットの確認
    * 写真撮影及び画像のアップロード
2. 画像にラベリングを行い学習させるインプットを整備する
    * S3バケットへ画像の格納
    * ラベリングを実施
    * ラベリング結果をモデル作成用に加工
3. FineTuneを利用し、学習インプットにてモデルの作成を行う
    * 学習用インプットデータを指定してモデルの作成を実施
4. モデルを起動する
    * 作成モデルを起動
5. 起動したモデルにてテスト判定を行う
    * エンドポイントを利用しテストイメージで判定させる
6. 疑似ベルトコンベアープログラムで判定させる
    * 完成したモデルを利用し最終テストを行う

In [ ]:
!apt-get update && apt-get upgrade -y
!apt-get install libgl1-mesa-dev -y
!pip install boto3
!pip install opencv-python pyheif

import sagemaker, json, numpy as np, os, boto3, uuid
from PIL import Image, ImageDraw, ImageOps, ImageColor, ImageFont
import matplotlib.patches as patches
from matplotlib import pyplot as plt
from glob import glob
import cv2
import pyheif
np.random.seed(seed=1234)
print('---------- 終 了 ----------')

## 1. モデル学習用の画像を撮影する
### S3バケットの確認
ユーザー毎に作成されている、S3バケットが存在している事を確認する

### 写真撮影及び画像のアップロード
スマホにて画像撮影を実施し、該当イメージをアップロードする
* Studioのアップロードアイコンをクリックしtrain_raw_imagesフォルダへアップロードする
* ファイルフォーマットの変換及び適正サイズへリサイズを実施する

In [ ]:
# format変換及びリサイズ
## アスペクト比を固定して、幅が指定した値になるようリサイズする。
def scale_to_width(img, width):
    height = round(img.height * width / img.width)
    return img.resize((width, height))

def convHeic2Png(image_path):
    new_name = image_path.replace('HEIC', 'png')
    heif_file = pyheif.read(image_path)
    data = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
        )
    # 画像をリサイズする
    data_resized = scale_to_width(data, 500)
    data_resized.save(new_name, "PNG")
    print(f'output heic -> png file:{new_name}')
    
def convJpeg2Png(image_path):
    new_name = image_path.replace('jpeg', 'png')
    img = Image.open(image_path)
    # 画像をリサイズする
    img_resized = scale_to_width(img, 500)
    img_resized.save(new_name, "PNG")
    print(f'output jpeg -> png file:{new_name}')

def convPng2Png(image_path):
    new_name = image_path
    img = Image.open(image_path)
    # 画像をリサイズする
    img_resized = scale_to_width(img, 500)
    img_resized.save(new_name, "PNG")
    print(f'output png -> png file:{new_name}')

print('データ変換開始')
lst = glob("./train_raw_images/*.*")
for l in lst:
    img = Image.open(l)
    if l.endswith('heic'):
        convHeic2Png(l)
        os.remove(l)
    elif l.endswith('jpeg'):
        convJpeg2Png(l)
        os.remove(l)
    elif l.endswith('png'):
        convPng2Png(l)
        os.remove(l)
    else:
        print('ファイルタイプエラー')
print('データ変換終了')


* パラーメーターを指定する

In [ ]:
# 各自のS3バケットを指定する
S3_BUCKET='training-user1-s3'
# 【変更不要】imageファイルの格納場所指定
IMAGES_PATH='training2/images'

* S3へアップロードする

In [ ]:
!aws s3 cp ./train_raw_images s3://{S3_BUCKET}/{IMAGES_PATH} --exclude ".*"  --recursive

## 2.画像にラベリングを行い学習させるインプットを整備する
* ラベリングチームの作成
グループメンバーにてラベリングチームを作成する
*
![](./images_for_ipynb/t2-labeling-1.png)
*
![](./images_for_ipynb/t2-labeling-2.png)
*
![](./images_for_ipynb/t2-labeling-3.png)
* ラベリングジョブを作成する
    * Ground Truth>ラベリングジョブ を選択
    * ラベリングジョブの作成 を押下する
    * 内容を入力し実行する
    
    ![](./images_for_ipynb/t2-labeling-4.png)
    
    * タスクのタイプを指定する
    
    ![](./images_for_ipynb/t2-labeling-5.png)
    
    * ワーカーの選択とツールの設定
    
    ![](./images_for_ipynb/t2-labeling-6.png)
    
    
* ラベリングを実施する
    * メールが来ているのでメールのリンクに従って、ツールを開いてラベリングを行う。
    * ラベリングが終わったことを確認する
* ラベリング結果をモデル作成用に加工
次のスクリプトを実行し各自用のS3バケットへ変換した結果をアップロードする

In [ ]:
# ラベリングを行ったジョブ名を指定してください
JOB_NAME="team-z-job2"

In [ ]:
OUTPUT_MANIFEST_PATH=''.join([IMAGES_PATH,'/',JOB_NAME,'/manifests/output/output.manifest'])
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(S3_BUCKET)
output_manifest_object = my_bucket.Object(OUTPUT_MANIFEST_PATH)
output_manifest_json = output_manifest_object.get()["Body"].read().decode('utf-8').split()
OUTPUT_PATH = ''.join(['training2/','annotations.json'])


# 結果のきのこの山やたけのこの里の位置情報を格納する辞書 
annotation_dict = {
    'images':[],
    'annotations':[]
}

# 画像のファイル名に使う一意なシーケンス番号
IMAGE_ID = 0

# ラベリング結果の行数分ループする
# ラベリング結果は 1 行につき 1 画像格納される
for manifest_line in output_manifest_json:
    # 画像のラベリング結果の読み込み
    manifest_dict = json.loads(manifest_line)
    # print(manifest_dict)
    # 画像のファイル名取得(ラベリング結果に格納されている)
    filename = manifest_dict['source-ref'].split('/')[-1]
    annotation_list = manifest_dict[JOB_NAME]['annotations']
    # print(annotation_list)
    # ラベリング結果を出力用辞書に格納
    annotation_dict['images'].append(
        {
            'file_name' : filename,
            'height' : manifest_dict[JOB_NAME]['image_size'][0]['height'],
            'width' : manifest_dict[JOB_NAME]['image_size'][0]['width'],
            'id' : IMAGE_ID
        }
    )
    for annotation in annotation_list:
        # 座標変換
        left = annotation['left']
        top = annotation['top']
        right = annotation['left'] + annotation['width']
        bottom = annotation['top'] + annotation['height']
        # アノテーションを編集
        annotation_dict['annotations'].append(
            {
                'image_id': IMAGE_ID,
                'bbox': [left, top, right, bottom],
                # 文字列へ変換し格納
                # 'category_id': manifest_dict['kinotake-user1-metadata']['class-map'][str(annotation['class_id'])]
                'category_id': annotation['class_id']
            }
        )
    
    IMAGE_ID += 1
    
obj = s3.Object(S3_BUCKET, OUTPUT_PATH)
obj.put(Body = json.dumps(annotation_dict, ensure_ascii=False)) #←変数をJSON変換し S3にPUTする

# 3. FineTuneを利用し、学習インプットにてモデルの作成を行う
## 学習用インプットデータを指定してモデルの作成を実施
* JumpStartを利用し、学習用インプットイメージで転移学習を行う
![](./images_for_ipynb/t2-model-1.png)
* 学習用インプットイメージで転移学習を行う
![](./images_for_ipynb/t2-model-2.png)
* 学習用インプットイメージで転移学習を行う
![](./images_for_ipynb/t2-model-3.png)


In [ ]:
# エンドポイントの名前
ENDPOINT_NAME='jumpstart-ftc-team-z-endpoint-1'
# 推論する画像の場所
TEST_IMAGE_FILE = 'test_raw_images/lattice.jpg'

In [ ]:
# 推論エンドポイントにアクセスするための sagemaker-runtime クライアントの生成
smr_client = boto3.client('sagemaker-runtime')

# 推論対象の画像を開いて変数に格納
with open(TEST_IMAGE_FILE, 'rb') as f:
    img_bin = f.read()

# 推論を実行
response = smr_client.invoke_endpoint(EndpointName=ENDPOINT_NAME, ContentType='application/x-image', Body=img_bin)

# 推論結果を読み込む
model_predictions = json.loads(response['Body'].read())
# 結果を可視化
# テスト画像を PIL を通して numpy array として開く
image_np = np.array(Image.open(TEST_IMAGE_FILE))
# matplotlibで描画する
fig = plt.figure(figsize=(20,20))
ax = plt.axes()
ax.imshow(image_np)
# 推論結果を変数に展開
bboxes, classes, confidences = model_predictions['normalized_boxes'], model_predictions['classes'], model_predictions['scores']
# 物体検出結果を検出した分だけループする
for idx in range(len(bboxes)):
    # 信頼度スコアが 0.5 以上のみ可視化する　★信頼度スコアを制限したい場合はここを変更
    if confidences[idx]>0.5:
        # 検出した座標（左上を(0,0),右下を(1,1)とした相対座標)を取得
        left, bot, right, top = bboxes[idx]
        # 相対座標を絶対座標に変換する
        x, w = [val * image_np.shape[1] for val in [left, right - left]]
        y, h = [val * image_np.shape[0] for val in [bot, top - bot]]
        # 検出した物体の ID を take/kino に読み替える
        class_name = 'kino' if int(classes[idx])==0 else 'take'
        # take/kinoに対して矩形で描画するための色を設定する
        color = 'blue' if class_name == 'take' else 'red'
        # matplotlib に検出した物体に矩形を描画する
        rect = patches.Rectangle((x, y), w, h, linewidth=3, edgecolor=color, facecolor='none')
        ax.add_patch(rect)
        # 左上に検出結果と信頼度スコアを描画する
        ax.text(x, y, "{} {:.0f}%".format(class_name, confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
fig

### 9-2. ベルトコンベアを模した推論
* `./test_raw_images/takenoko.jpg` にタケノコが横一列に並んでいる（１つだけキノコが混在）
* 512x512の画像をスライドしながら切り出すことでベルトコンベアでお菓子が流れているような動画として扱う
* 各画像に対して推論をかけ、最後に１つの動画として出力する

In [ ]:
%%time
# 画像を切り出すためのコード
# 開始地点設定
x ,y = 0,320
# 切り出すサイズ設定
CROP_SIZE=(512,512)
# 切り出す対象の画像を PIL で開く
img = Image.open('./test_raw_images/takenoko.jpg')
# 切り出した画像を保存するディレクトリ
CROP_DIR = './test_crop_images/'
# re-run 用の削除コマンド
!rm -rf {CROP_DIR}/*.png
# 1pxずらしてループ
count = 1
max_count = img.size[0]-CROP_SIZE[0]
for i in range(max_count):
    # 画像の切り出し
    crop_img = img.crop((i,y,i+CROP_SIZE[0],y+CROP_SIZE[1]))
    # 切り出した画像を保存
    file_name = f'{CROP_DIR}{str(i).zfill(5)}.png'
    crop_img.save(file_name)
    print("\r"+str(count)+"/"+str(max_count),end="")
    count += 1
print('終了')

In [ ]:
def imgInference(img_file_path):
    # 切り出した画像を開く
    with open(img_file_path,'rb') as f:
        img_bin = f.read()
    # 推論エンドポイントに画像を投げる
    response = smr_client.invoke_endpoint(EndpointName=ENDPOINT_NAME, ContentType='application/x-image', Body=img_bin)
    # 推論結果を読み込む
    pred=json.loads(response['Body'].read())
    # print(pred)
    # 推論結果を展開
    bboxes, classes, confidences = pred['normalized_boxes'], pred['classes'], pred['scores']
    # print(confidences)
    # 切り出した画像を PIL で開く
    img = Image.open(img_file_path)
    # 矩形やテキストを描くために draw インスタンスを生成
    draw = ImageDraw.Draw(img)
    # 検出したkino/take分ループ
    for i in range(len(bboxes)):
        # 信頼度スコアが0.8以上のみ描画する
        if confidences[i]>0.7:
            # 矩形の相対座標を取得
            left, top, right, bottom = bboxes[i]
            # 矩形の相対座標を絶対座標に変換
            left = img.size[0] * left
            top = img.size[1] * top
            right = img.size[0] * right
            bottom = img.size[1] * bottom
            # 検出した物体の ID を take/kino に読み替える
            text = 'take' if int(classes[i])==1 else 'kino'
            # take/kinoに対して矩形で描画するための色を設定する
            color = 'blue' if text == 'take' else 'red'
            # 矩形の左上に表示する文字の大きさを設定、きのこの山なら大きくする
            TEXTSIZE=14 if classes[i]=='1' else 18
            # 矩形の先の太さを設定、きのこの山なら太くする
            LINEWIDTH=4 if classes[i]=='1' else 6
            # 矩形を描画する
            draw.rectangle([(left,top),(right,bottom)], outline=color, width=LINEWIDTH)
            # 矩形の左上に描画する信頼度スコアの取得
            text += f' {str(round(confidences[i],3))}'
            # テキストを描画する場所を取得
            txpos = (left, top-TEXTSIZE-LINEWIDTH//2)
            # フォントの設定
            font = ImageFont.truetype("/usr/share/fonts/truetype/noto/NotoMono-Regular.ttf", size=TEXTSIZE)
            # 描画するテキストのサイズを取得
            bbox = draw.textbbox(txpos, text, font=font)
            # テキストの背景用の矩形を描画
            draw.rectangle(bbox, outline=color, fill=color, width=LINEWIDTH)
            # テキストを描画
            draw.text(txpos, text, fill='white',font=font)
    return img


# テスト実行
img_file_path = glob(f'{CROP_DIR}*.png')[0]
ret = imgInference(img_file_path)
image_np = np.array(ret)
## matplotlibで描画する
fig = plt.figure(figsize=(20,20))
ax = plt.axes()
ax.imshow(image_np)
fig

全データ推論して、矩形を描いた画像を生成する

In [ ]:
%%time
# 検出結果を保存するディレクトリを設定
DETECT_DIR='./test_detect_images/'
# re-run 用の削除コマンド
!rm -rf {DETECT_DIR}/*.png

count = 1
# 切り出した画像分だけループ
for img_file_path in sorted(glob(f'{CROP_DIR}*.png')):
    img = imgInference(img_file_path)
    # 画像をファイルに書き出す
    img.save(img_file_path.replace(CROP_DIR,DETECT_DIR))
    print("\r"+str(count),end="")
    count += 1

In [ ]:
# 書き出した画像を連結して動画にする    
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
video = cv2.VideoWriter('./video.mp4',fourcc, 120.0, CROP_SIZE)
for img_file_path in sorted(glob(f'{DETECT_DIR}*.png')):
    img = cv2.imread(img_file_path)
    video.write(img)
video.release()